## 1. Carga y unificación de datos

In [2]:

import pandas as pd
df_contract = pd.read_csv('/datasets/final_provider/contract.csv')
df_personal = pd.read_csv('/datasets/final_provider/personal.csv')
df_internet = pd.read_csv('/datasets/final_provider/internet.csv')
df_phone = pd.read_csv('/datasets/final_provider/phone.csv')


In [3]:
df_contract.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   BeginDate         7043 non-null   object 
 2   EndDate           7043 non-null   object 
 3   Type              7043 non-null   object 
 4   PaperlessBilling  7043 non-null   object 
 5   PaymentMethod     7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7043 non-null   object 
dtypes: float64(1), object(7)
memory usage: 440.3+ KB


In [5]:
df_personal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     7043 non-null   object
 1   gender         7043 non-null   object
 2   SeniorCitizen  7043 non-null   int64 
 3   Partner        7043 non-null   object
 4   Dependents     7043 non-null   object
dtypes: int64(1), object(4)
memory usage: 275.2+ KB


In [6]:
df_internet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5517 entries, 0 to 5516
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customerID        5517 non-null   object
 1   InternetService   5517 non-null   object
 2   OnlineSecurity    5517 non-null   object
 3   OnlineBackup      5517 non-null   object
 4   DeviceProtection  5517 non-null   object
 5   TechSupport       5517 non-null   object
 6   StreamingTV       5517 non-null   object
 7   StreamingMovies   5517 non-null   object
dtypes: object(8)
memory usage: 344.9+ KB


In [7]:
df_phone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6361 entries, 0 to 6360
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     6361 non-null   object
 1   MultipleLines  6361 non-null   object
dtypes: object(2)
memory usage: 99.5+ KB


In [8]:
df = df_contract.merge(df_personal, on='customerID', how='left')
df = df.merge(df_internet, on='customerID', how='left')
df = df.merge(df_phone, on='customerID', how='left')

df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   BeginDate         7043 non-null   object 
 2   EndDate           7043 non-null   object 
 3   Type              7043 non-null   object 
 4   PaperlessBilling  7043 non-null   object 
 5   PaymentMethod     7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7043 non-null   object 
 8   gender            7043 non-null   object 
 9   SeniorCitizen     7043 non-null   int64  
 10  Partner           7043 non-null   object 
 11  Dependents        7043 non-null   object 
 12  InternetService   5517 non-null   object 
 13  OnlineSecurity    5517 non-null   object 
 14  OnlineBackup      5517 non-null   object 
 15  DeviceProtection  5517 non-null   object 
 16  TechSupport       5517 non-null   object 


El conjunto de datos principal se construyó utilizando la tabla contract como base, ya que contiene la información de cancelación del cliente. Las tablas personal, internet y phone se unieron mediante la columna customerID usando uniones tipo left join.

Tras la unificación, se observan valores faltantes en las variables relacionadas con servicios de Internet y telefonía, lo cual es consistente con clientes que no cuentan con dichos servicios y no corresponde a errores en los datos. Estos valores se tratarán adecuadamente durante la etapa de preprocesamiento.

## 2. Análisis exploratorio de datos (EDA)

In [10]:
# variable objetivo (EndDate)

df['EndDate'].value_counts()


No                     5174
2019-11-01 00:00:00     485
2019-12-01 00:00:00     466
2020-01-01 00:00:00     460
2019-10-01 00:00:00     458
Name: EndDate, dtype: int64

In [11]:
df['EndDate'].value_counts(normalize=True)


No                     0.734630
2019-11-01 00:00:00    0.068863
2019-12-01 00:00:00    0.066165
2020-01-01 00:00:00    0.065313
2019-10-01 00:00:00    0.065029
Name: EndDate, dtype: float64

In [12]:

# Creación preliminar del target
df['churn'] = (df['EndDate'] != 'No').astype(int)
df['churn'].value_counts(normalize=True)



0    0.73463
1    0.26537
Name: churn, dtype: float64

In [14]:
#Revisión de TotalCharges

df['TotalCharges'].head()

0      29.85
1     1889.5
2     108.15
3    1840.75
4     151.65
Name: TotalCharges, dtype: object

In [15]:
df['TotalCharges'].value_counts().head(10)


         11
20.2     11
19.75     9
19.65     8
20.05     8
19.9      8
45.3      7
19.55     7
19.45     6
20.15     6
Name: TotalCharges, dtype: int64

In [16]:
df['TotalCharges'].isna().sum()


0

In [17]:

df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'].isna().sum()



11

In [19]:
# Análisis de variables numéricas
df[['MonthlyCharges', 'TotalCharges']].describe()


,MonthlyCharges,TotalCharges
count,7043.000000,7032.000000
mean,64.761692,2283.300441
std,30.090047,2266.771362
min,18.250000,18.800000
25%,35.500000,401.450000
50%,70.350000,1397.475000
75%,89.850000,3794.737500
max,118.750000,8684.800000


In [21]:
df.groupby('churn')[['MonthlyCharges', 'TotalCharges']].mean()

,MonthlyCharges,TotalCharges
churn,,
0,61.265124,2555.344141
1,74.441332,1531.796094


In [22]:

# Variables categóricas clave vs churn
pd.crosstab(df['Type'], df['churn'], normalize='index')


churn,0,1
Type,,
Month-to-month,0.572903,0.427097
One year,0.887305,0.112695
Two year,0.971681,0.028319


In [23]:
pd.crosstab(df['PaperlessBilling'], df['churn'], normalize='index')


churn,0,1
PaperlessBilling,,
No,0.836699,0.163301
Yes,0.664349,0.335651


In [24]:
pd.crosstab(df['PaymentMethod'], df['churn'], normalize='index')


churn,0,1
PaymentMethod,,
Bank transfer (automatic),0.832902,0.167098
Credit card (automatic),0.847569,0.152431
Electronic check,0.547146,0.452854
Mailed check,0.808933,0.191067


In [26]:
# Servicios de Internet y churn 

internet_cols = [
    'InternetService', 'OnlineSecurity', 'OnlineBackup',
    'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies'
]

for col in internet_cols:
    print(col)
    display(pd.crosstab(df[col], df['churn'], normalize='index'))



InternetService


churn,0,1
InternetService,,
DSL,0.810409,0.189591
Fiber optic,0.581072,0.418928


OnlineSecurity


churn,0,1
OnlineSecurity,,
No,0.582333,0.417667
Yes,0.853888,0.146112


OnlineBackup


...

churn,0,1
TechSupport,,
No,0.583645,0.416355
Yes,0.848337,0.151663


StreamingTV


churn,0,1
StreamingTV,,
No,0.664769,0.335231
Yes,0.699298,0.300702


StreamingMovies


churn,0,1
StreamingMovies,,
No,0.663196,0.336804
Yes,0.700586,0.299414


## 3. Preprocesamiento y preparación de datos

In [27]:
df['target'] = (df['EndDate'] == 'No').astype(int)
df['target'].value_counts(normalize=True)


1    0.73463
0    0.26537
Name: target, dtype: float64

In [28]:
df_model = df.drop(columns=['customerID', 'EndDate', 'BeginDate'])


In [31]:
df_model['TotalCharges'] = df_model['TotalCharges'].fillna(
    df_model['TotalCharges'].median()
)
service_cols = [
    'InternetService', 'OnlineSecurity', 'OnlineBackup',
    'DeviceProtection', 'TechSupport', 'StreamingTV',
    'StreamingMovies', 'MultipleLines'
]

df_model[service_cols] = df_model[service_cols].fillna('No')


In [49]:
num_cols = ['MonthlyCharges', 'TotalCharges', 'SeniorCitizen']
cat_cols = df_model.drop(columns=num_cols + ['target']).columns

df_encoded = pd.get_dummies(
    df_model,
    columns=cat_cols,
    drop_first=True
)


In [54]:

# 1. Eliminar columnas que generan fuga de información
df_model = df.drop(columns=['EndDate', 'customerID', 'churn'])

# 2. Corregir TotalCharges
df_model['TotalCharges'] = pd.to_numeric(df_model['TotalCharges'], errors='coerce')

# 3. Separar target
y = df_model['target']
X = df_model.drop(columns=['target'])

# 4. One-Hot Encoding
X = pd.get_dummies(X, drop_first=True)

# 5. Eliminar filas con NaN
X = X.loc[X.notna().all(axis=1)]
y = y.loc[X.index]


In [55]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42,
    stratify=y
)


## 4. Entrenamiento y evaluación de modelos

In [56]:
# Modelo 1

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score

log_reg = LogisticRegression(
    max_iter=1000,
    random_state=42
)

log_reg.fit(X_train, y_train)

y_pred_proba = log_reg.predict_proba(X_test)[:, 1]
y_pred = log_reg.predict(X_test)

auc_log = roc_auc_score(y_test, y_pred_proba)
acc_log = accuracy_score(y_test, y_pred)

auc_log, acc_log


(0.8561661444658043, 0.800910125142207)

In [57]:
# Modelo 2 
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)

y_pred_proba_rf = rf.predict_proba(X_test)[:, 1]
y_pred_rf = rf.predict(X_test)

auc_rf = roc_auc_score(y_test, y_pred_proba_rf)
acc_rf = accuracy_score(y_test, y_pred_rf)

auc_rf, acc_rf


(0.8755185379923934, 0.8469852104664392)

In [58]:
# Comparación de modelos

results = pd.DataFrame({
    'Model': ['Logistic Regression', 'Random Forest'],
    'AUC-ROC': [auc_log, auc_rf],
    'Accuracy': [acc_log, acc_rf]
})

results


,Model,AUC-ROC,Accuracy
0,Logistic Regression,0.856166,0.800910
1,Random Forest,0.875519,0.846985


El modelo Random Forest alcanzó un AUC-ROC de 0.876, superando el umbral requerido para una solución de alta calidad. Este resultado indica que el modelo discrimina eficazmente entre clientes que continuarán y aquellos con mayor riesgo de cancelación, lo que lo hace adecuado para su uso en estrategias de retención.

## 5. Análisis de resultados y conclusiones

Se entrenaron y evaluaron dos modelos de clasificación para predecir la cancelación de clientes (churn): Regresión Logística y Random Forest.
La métrica principal utilizada fue AUC-ROC, complementada con exactitud (accuracy), de acuerdo con los criterios del proyecto.

La Regresión Logística obtuvo un AUC-ROC de 0.856 y una exactitud de 0.801. Este desempeño indica que el modelo es capaz de diferenciar adecuadamente entre clientes que permanecerán y aquellos que cancelarán el servicio, funcionando como un baseline sólido e interpretable. Su simplicidad permite comprender la influencia de las variables, aunque su capacidad para capturar relaciones complejas es limitada.

Por su parte, el modelo Random Forest presentó un mejor rendimiento general, alcanzando un AUC-ROC de 0.876 y una exactitud de 0.847. Este resultado demuestra una mayor capacidad de discriminación, lo que sugiere que el modelo captura relaciones no lineales e interacciones entre variables como el tipo de contrato, los cargos mensuales y los servicios adicionales contratados. De acuerdo con los criterios de evaluación, este desempeño se encuentra dentro del rango de modelos de alta calidad.

En ambos casos, los resultados son coherentes con el análisis exploratorio de datos, donde se identificó una mayor tasa de cancelación en contratos mensuales, clientes con facturación electrónica y usuarios de servicios de fibra óptica sin servicios adicionales de soporte o seguridad.

### Conclusiones


El uso de modelos de aprendizaje automático permite predecir de forma efectiva la cancelación de clientes en Interconnect.
El modelo Random Forest fue seleccionado como la mejor solución, ya que alcanzó el mayor valor de AUC-ROC y exactitud, superando el umbral requerido para una solución de alto desempeño.

- Este modelo puede ser utilizado por el equipo de marketing para:

- Identificar clientes con alto riesgo de cancelación.

Priorizar la aplicación de promociones, descuentos o planes especiales.

Optimizar recursos al enfocar las acciones de retención en los clientes más propensos a abandonar el servicio.

Como trabajo futuro, se podrían explorar técnicas de ajuste de hiperparámetros, análisis de importancia de variables y validación temporal para mejorar aún más el desempeño del modelo y su aplicabilidad en un entorno productivo.